In [1]:
# 所有需要导入的库

import re
import ast
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import load_model

Using TensorFlow backend.


In [2]:
# 导入数据

word_dict = pd.read_csv('word_dict.csv', encoding='utf8')
word_dict = word_dict.drop(['0'], axis=1)
word_dict.columns = ['0', 'id']

import pickle
def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

int_catagory = load_obj('int_catagory')
catagory_dict = load_obj('catagory_dict')

In [4]:
# 导入模型

model = load_model('model.hdf5')

In [5]:
maxlen=25

# 预测函数

def predict_(title):
    words = re.findall('[\x80-\xff]{3}|[\w\W]', title)
    w2v = [word_dict[word_dict['0']==x]['id'].values[0] for x in words]
    xn = sequence.pad_sequences([w2v], maxlen=maxlen)
    predicted = model.predict_classes(xn, verbose=0)[0]
    return int_catagory[predicted]

# 前三种可能性分类

def predict_3(title):
    words = re.findall('[\x80-\xff]{3}|[\w\W]', title)
    w2v = [word_dict[word_dict['0']==x]['id'].values[0] for x in words]
    xn = sequence.pad_sequences([w2v], maxlen=maxlen)
    predicted = model.predict(xn, verbose=0)[0]
    predicted_sort = predicted.argsort() 
    li = [(int_catagory[p], predicted[p]*100) for p in predicted_sort[-3:]]
    return li[::-1]

In [6]:
# 拉取当天新闻并预测, 测试准确率

def check_news_today():
    today = datetime.today().strftime("%Y-%m%d")
    year, month_day = today.split('-')
    base_url = "http://www.chinanews.com/scroll-news/%s/%s/news.shtml" % (year, month_day)
    resp = requests.get(base_url, timeout=10)
    resp.encoding = "gbk"
    soup = BeautifulSoup(resp.text, "html.parser")
    content = soup.find('div', class_='content_list')
    li = content.find_all("li")
    count = 0
    match = 0
    for item in li:
        try:
            catagory = item.find('div', class_='dd_lm').text.replace(r'[', '').replace(r']', '')
            if catagory in ['', u'图片', u'视频', u'报摘']:
                continue
            title = item.find('div', class_='dd_bt').text
            href = item.find('div', class_='dd_bt').a.attrs['href']
            prediction = predict_(title)
            print "[%s] prediction:[%s] %s" % (catagory, prediction, title)
            if catagory == prediction:
                match += 1
            count += 1
        except:
            continue
    print "acc: %.2f%% (%d/%d)" % (float(100*match)/count, match, count)
    
def check_news_today_3():
    today = datetime.today().strftime("%Y-%m%d")
    year, month_day = today.split('-')
    base_url = "http://www.chinanews.com/scroll-news/%s/%s/news.shtml" % (year, month_day)
    resp = requests.get(base_url, timeout=10)
    resp.encoding = "gbk"
    soup = BeautifulSoup(resp.text, "html.parser")
    content = soup.find('div', class_='content_list')
    li = content.find_all("li")
    count = 0
    match = 0
    for item in li:
        try:
            catagory = item.find('div', class_='dd_lm').text.replace(r'[', '').replace(r']', '')
            if catagory in ['', u'图片', u'视频', u'报摘']:
                continue
            title = item.find('div', class_='dd_bt').text
            href = item.find('div', class_='dd_bt').a.attrs['href']
            prediction = predict_3(title)
            print "[%s] prediction:[%s(%.2f%%) %s(%.2f%%) %s(%.2f%%)] %s" % \
                (catagory, prediction[0][0], prediction[0][1], prediction[1][0], prediction[1][1], prediction[2][0], prediction[2][1], title)
            if catagory in [prediction[0][0], prediction[1][0], prediction[2][0]]:
                match += 1
            count += 1
        except:
            continue
    print "acc: %.2f%% (%d/%d)" % (float(100*match)/count, match, count)

In [7]:
check_news_today()

[国内] prediction:[国内] 中方启动人道主义紧急援助助菲律宾应对台风袭击
[社会] prediction:[社会] 维卡币特大网络传销案二审宣判 涉案金额达16亿
[国内] prediction:[国内] 国务院明确环境保护税收入归属：全部作为地方收入
[国内] prediction:[国内] 食药监总局：鼓励中小学食堂将明厨亮灶视频传云平台
[社会] prediction:[社会] 动粗不划算！河南警方张贴“打架成本清单”获点赞
[体育] prediction:[娱乐] 一年坎坷也曾怀疑自己，如今五冠在手的柯洁将再战AI
[社会] prediction:[文化] 海内外佛教界追思中国佛教泰斗一诚长老（图）
[财经] prediction:[财经] 俄罗斯成中国最大陆路口岸进出口“老大”
[社会] prediction:[地方] 福建周宁老区村的扶贫路：抱团发展促农民增收
[国际] prediction:[国际] 俄罗斯首都一工厂发生枪击 当局否认有人被劫持
[财经] prediction:[I  T] 海南发起成立“大数据产业联盟”
[文化] prediction:[文化] 中国首个万里茶道文物展首展 再现昔日国际商道繁荣风貌
[台湾] prediction:[台湾] “海峡两岸产业合作区”落户四川 
[台湾] prediction:[财经] 小电峰值日订单破75万 市占率稳居共享充电行业第一
[财经] prediction:[国内] 上海市长应勇：今年上海经济稳中向好 好于预期
[华人] prediction:[华人] 意大利米兰中国文化中心首次在河北“落地” 
[财经] prediction:[财经] 2018年中国民航拟开工重点项目33个 固定资产投资860亿元
[国际] prediction:[金融] 对央行秋后算账之年
[国内] prediction:[社会] 中国出严规治理校园欺凌 涉嫌违法犯罪将以公检法为主处置
[汽车] prediction:[汽车] 总体产能规划已超500万辆 新能源车产能过剩现苗头
[汽车] prediction:[I  T] 阿里巴巴高管调整 蒋凡任淘宝总裁 靖捷任天猫总裁
[金融] prediction:[金融] 中国累计发行绿色债券近4000亿元 官方要求完善绿债认证制度
[汽车] prediction:[汽

[社会] prediction:[社会] 老人晕倒路边口吐白沫 幸遇医生及时施救
[财经] prediction:[财经] 经常点外卖的必看：这个潜规则实在太坑人
[国内] prediction:[国内] 中共中央政治局召开会议 决定召开十九届二中全会
[I  T] prediction:[财经] 2017产业大数据应用会议在北京举办
[体育] prediction:[体育] 邹市明的艰难时刻：眼睛旧伤复发、站到退役边缘
[华人] prediction:[华人] 华裔工程师发明快煲锅风靡北美 有多种烹饪功能
[社会] prediction:[社会] 高速扎胎盗窃团伙落网：啤酒盖上放尖钢管布陷阱
[社会] prediction:[法治] 黑客入侵网站添加建造师名单 伪造证件谋利被抓
[华人] prediction:[房产] 西班牙给年轻人一万欧元的买房补贴暂缓施行
[国内] prediction:[国内] 中国“北斗”开通五年 性能稳定走出国门
[国际] prediction:[国际] 暴雪袭日本北海道致多人伤 引列车脱轨航班停飞
[台湾] prediction:[台湾] 台湾青年讲述大陆生活：这里充满机会 我应该早些来
[社会] prediction:[社会] 零下4℃大学生跳灌渠救人 十几位同学参与救援
[国内] prediction:[国内] 珠江委：澳门等地元旦期间供水可有效保障
[国内] prediction:[I  T] 云狐时代落户重庆璧山 打造高新领域科技创新产业基地
[财经] prediction:[国内] 成振宇：民的根在于家 企业的根却是国
[国际] prediction:[国际] 瑞士一客船在德国撞上高架桥桥柱 造成25人受伤 
[国际] prediction:[国际] 过渡年抑或转折年？2017俄罗斯内政外交新特征
[财经] prediction:[房产] 钉钉注册用户过亿 社交格局正在变化
[财经] prediction:[I  T] 多牛星游合体估值或超20亿 改变泛娱乐媒体行业格局
[生活] prediction:[生活] 海南迎元旦旅游小高峰 多家景区推出优惠
[社会] prediction:[财经] 中国移动捆绑支付渠道遭疑：充话费非得用“和包”？
[社会] prediction:[健康] 中国平安携手重庆首创“AI+大数据”疾病预

[华人] prediction:[华人] 乌兹别克斯坦出台旅游签证新规 简化外钞出入境规定
[社会] prediction:[社会] 一人偷油一人望风 山东抓获夫唱妇随“油耗子”
[财经] prediction:[财经] 1-11月全国规模以上工业企业利润同比增21.9%
[国际] prediction:[国际] 日本外相分别会晤巴以首脑 称支持“两国方案”
[国际] prediction:[国际] 巴基斯坦粉红出租车将扩大队伍 为女性增加便利(图)
[财经] prediction:[I  T] 如果贾跃亭不回来 我们还能做什么？
[社会] prediction:[社会] 买礼品见家长“一条龙” 急“脱单”也要当心陷阱
[财经] prediction:[健康] 医疗行业明年迎风而立？ 共享医疗创新药齐被看好
[娱乐] prediction:[娱乐] 央视推出元旦特别节目《山水中国美》
[财经] prediction:[I  T] 铁甲与远东宏信合作联手打造工程机械互联网生态链
[国际] prediction:[国际] 日本筱山市收到1亿日元匿名捐款 希望其更改市名
[体育] prediction:[体育] 2017中国体坛十大热项：姚明“怼”中国篮球旧风气
[华人] prediction:[国际] 美加州山火近9成受遏制 撤离令取消并对游客开放
[国内] prediction:[文化] 2017诺奖物理奖得主：发现引力波 一场40年的“打赌”
[金融] prediction:[金融] 可转债发行井喷 投资者不再“贪图”打新收益
[文化] prediction:[教育] 2017全国艺术院校教育创新发展院(校)长论坛举行
[文化] prediction:[房产] 张裕酿酒公司入选第一批国家工业遗产名单
[国内] prediction:[国内] 邵新宇任华中科技大学党委书记 路钢不再担任
[体育] prediction:[体育] 20岁柯洁斩获个人第五个世界冠军 打破李昌镐纪录
[生活] prediction:[社会] 评论：精神疾病治疗不适合成资本淘金的富矿
[社会] prediction:[社会] 孩子沦为父母牟利工具 检方异地督促撤销监护权
[财经] prediction:[财经] 让贾跃亭回国，到底有啥办法？
[国际] prediction:[国际] 新加坡年轻

In [8]:
check_news_today_3()

[国内] prediction:[国内(72.64%) 国际(24.33%) 军事(2.05%)] 中方启动人道主义紧急援助助菲律宾应对台风袭击
[社会] prediction:[社会(36.37%) 金融(24.47%) 法治(21.14%)] 维卡币特大网络传销案二审宣判 涉案金额达16亿
[国内] prediction:[国内(58.97%) 能源(24.06%) 财经(13.48%)] 国务院明确环境保护税收入归属：全部作为地方收入
[国内] prediction:[国内(53.73%) 社会(33.42%) 生活(4.09%)] 食药监总局：鼓励中小学食堂将明厨亮灶视频传云平台
[社会] prediction:[社会(63.64%) 法治(26.47%) 国内(5.75%)] 动粗不划算！河南警方张贴“打架成本清单”获点赞
[体育] prediction:[娱乐(86.34%) 文化(3.52%) I  T(1.75%)] 一年坎坷也曾怀疑自己，如今五冠在手的柯洁将再战AI
[社会] prediction:[文化(81.43%) 社会(6.42%) 生活(3.48%)] 海内外佛教界追思中国佛教泰斗一诚长老（图）
[财经] prediction:[财经(73.34%) 地方(8.17%) 国内(7.58%)] 俄罗斯成中国最大陆路口岸进出口“老大”
[社会] prediction:[地方(51.73%) 社会(39.39%) 财经(5.71%)] 福建周宁老区村的扶贫路：抱团发展促农民增收
[国际] prediction:[国际(99.67%) 军事(0.29%) 华人(0.02%)] 俄罗斯首都一工厂发生枪击 当局否认有人被劫持
[财经] prediction:[I  T(30.09%) 财经(30.06%) 社会(14.41%)] 海南发起成立“大数据产业联盟”
[文化] prediction:[文化(58.69%) 生活(9.69%) 财经(8.99%)] 中国首个万里茶道文物展首展 再现昔日国际商道繁荣风貌
[台湾] prediction:[台湾(97.72%) 财经(1.12%) 地方(0.44%)] “海峡两岸产业合作区”落户四川 
[台湾] prediction:[财经(26.28%) I  T(25.01%) 生活(23.40%)] 小电峰

[房产] prediction:[房产(94.06%) 财经(5.45%) 金融(0.20%)] 房企集体看好北京南城发展 万和斐丽等楼盘相继亮相
[体育] prediction:[I  T(67.42%) 财经(16.20%) 体育(5.74%)] 2017 MEST移动电竞大赛收官 咪咕多点创新助力竞技体育发展
[财经] prediction:[财经(33.60%) 社会(15.80%) 国际(13.89%)] 云集微店CEO肖尚略获“十大创新经济人物” 称号
[娱乐] prediction:[文化(79.91%) 娱乐(11.43%) 台湾(4.58%)] 王鹤润首次挑战古装形象 为新片角色剃光头
[文化] prediction:[娱乐(52.93%) 文化(42.87%) 财经(1.85%)] 首个国家级影视产业园揭牌 中国影视剧将“抱团出海”
[华人] prediction:[华人(98.60%) 国内(0.40%) 财经(0.24%)] “侨”这新时代：搭建中哈贸易新桥梁 促合作共赢
[华人] prediction:[文化(49.02%) 国际(48.46%) 华人(0.79%)] 美一博物馆13件名画失窃27年 千万美元悬赏月底截止
[华人] prediction:[汽车(85.18%) 证券(6.68%) 财经(4.98%)] 空调行业迎来新变局 TCL年产销量1000万套挺进前四
[文化] prediction:[文化(58.21%) 娱乐(35.06%) 国际(3.10%)] 纪录片《雪地求生》上线 讲述北极熊迁徙故事
[港澳] prediction:[法治(48.36%) 港澳(27.39%) 社会(18.50%)] 男子挪用客户购房款40万 在澳门狂赌一把输尽
[国际] prediction:[国际(89.05%) 娱乐(3.51%) 财经(2.39%)] 韩国平昌奥组委公布颁奖礼仪人员服装、奖品等元素
[华人] prediction:[国际(96.78%) 财经(1.54%) 华人(0.62%)] 外媒：研究称智利河流垃圾污染严重 亟需限塑令 
[台湾] prediction:[台湾(99.54%) 军事(0.32%) 社会(0.04%)] 台军金门营区发生车辆翻覆意外 致1死1伤
[国内] prediction:[国内(38.56%) 教

[社会] prediction:[社会(60.52%) 法治(21.50%) 国内(6.29%)] “幽灵”微信号建群传播淫秽视频5万余部 11名嫌犯落网
[社会] prediction:[社会(48.51%) 法治(24.80%) 生活(8.84%)] 哈尔滨“马桶刷刷茶杯”五星级酒店被依法查处
[国内] prediction:[社会(49.55%) 法治(31.00%) 国内(17.19%)] 内蒙古交通厅副厅长严洪波严重违纪被开除党籍
[台湾] prediction:[台湾(99.82%) 国内(0.15%) 社会(0.01%)] 国台办回应新党人士遭查、台方防务报告等涉台热点
[国际] prediction:[国际(85.91%) 生活(4.18%) 娱乐(3.42%)] 多米尼加发生假朗姆酒中毒事件 致十多人丧生
[国内] prediction:[国内(83.69%) 社会(13.90%) 能源(1.53%)] 中央环保督察组吉林反馈：环保工作说一套做一套
[社会] prediction:[社会(92.80%) 健康(4.02%) 法治(1.33%)] 扬州一医院儿科不堪重负医生累病 劝患者去其他医院
[国内] prediction:[能源(47.69%) 财经(33.29%) 国内(16.93%)] 环保税法明年1月1日起实施 明确环境保护税为地方收入
[社会] prediction:[社会(43.74%) 国内(19.76%) 财经(11.99%)] “千年古县”的社会治理“新创意”：农民夜校 功德银行 村落餐厅
[华人] prediction:[国际(39.66%) 军事(32.94%) 华人(26.14%)] 外媒：节日严防恐袭 悉尼繁华街道被大卡车封锁
[社会] prediction:[生活(46.82%) 社会(18.00%) 财经(14.23%)] 元旦将至你准备去哪里跨年？ 假日旅游指南供参考
[体育] prediction:[体育(98.76%) 娱乐(0.37%) 社会(0.25%)] 邹市明：病情依旧不容乐观，或将在医院继续治疗
[台湾] prediction:[台湾(99.64%) 国内(0.26%) 财经(0.03%)] 国台办：金门引水工程大陆段主体工程已经建成
[国内] prediction:[国内(46.79%) 国际(12

[文化] prediction:[文化(77.21%) 社会(11.73%) 地方(10.03%)] 温州永嘉唐代瓯窑遗址考古成果发布 发现唐代“官作”
[娱乐] prediction:[文化(89.73%) 娱乐(8.52%) 社会(0.80%)] 北京舞蹈大赛展示30年优秀成果 经典作品涌现
[华人] prediction:[华人(99.33%) 国际(0.64%) 财经(0.01%)] 美现金管理趋严 华人女子连续存现金账户遭关闭
[体育] prediction:[体育(99.99%) 国际(0.00%) 财经(0.00%)] 中超升班马北京人和：目标是先在中超站稳脚跟
[华人] prediction:[华人(68.63%) 国际(30.92%) 社会(0.12%)] 研究：美加州环评法恐遭滥用 成住房短缺帮凶
[国际] prediction:[国际(80.80%) 文化(10.72%) 华人(3.45%)] 使馆搬迁潮刚开始？以色列称10国酝酿效仿美国 
[国际] prediction:[I  T(46.74%) 财经(42.80%) 证券(4.89%)] 多家央企完成更名 电信业混改将成重头戏
[国际] prediction:[文化(50.64%) 国际(45.36%) 华人(1.84%)] 英国国家档案馆遗失数千份文件 含多份机密文件
[社会] prediction:[财经(55.43%) 社会(22.50%) 地方(8.84%)] 杭州首条完整开通试运营地铁线路诞生 手机能直接入闸
[体育] prediction:[体育(99.98%) 国际(0.01%) 财经(0.00%)] CBA下赛季启用职业裁判 将实现裁判全面职业化
[国内] prediction:[国内(69.45%) 教育(12.37%) 社会(10.93%)] 11部门印发《加强中小学生欺凌综合治理方案》
[国内] prediction:[财经(41.22%) 房产(33.94%) 生活(13.44%)] 王府井商圈加码儿童业态 能否唤回金街老情怀？
[国内] prediction:[国内(87.11%) 教育(6.65%) 社会(4.40%)] 11部门：学生欺凌事件处置以学校为主 须依法依规
[体育] prediction:[体育(100.00%) 国际(0.00%) 财经(0.00

[能源] prediction:[能源(33.38%) 财经(32.62%) 国内(26.97%)] 中国水电、风电、核电等在建规模稳居世界第一
[国际] prediction:[国际(98.81%) 财经(0.51%) 金融(0.17%)] 险些酿成重大事故 日本反思新干线安全检查制度
[国际] prediction:[国际(99.89%) 财经(0.04%) 国内(0.03%)] 韩国检方上门调查 朴槿惠三缄其口拒绝配合
[国际] prediction:[国际(99.93%) 华人(0.04%) 军事(0.02%)] 韩媒揭朴槿惠在拘留所生活：爱读书 无外部接触 
[社会] prediction:[法治(57.48%) 社会(36.96%) 地方(3.00%)] 男子杀妻被判死缓 服刑期数次伤人或改判死刑立即执行
[社会] prediction:[社会(84.76%) 法治(10.13%) 生活(2.89%)] 父母离婚试管婴儿该判给谁 讲情还是说理引发讨论
[华人] prediction:[华人(98.89%) 国际(0.60%) 国内(0.27%)] 2017年这些华侨华人将普通生活过得不平凡
[社会] prediction:[社会(86.43%) 国内(7.75%) 法治(3.35%)] 杭州保姆纵火案：被害人家属要求公开录音等信息
[国际] prediction:[国际(99.35%) 军事(0.28%) 华人(0.19%)] 普京被正式推举为总统候选人 促严防国外干涉势力
[国际] prediction:[国际(99.07%) 军事(0.86%) 华人(0.02%)] 普京要求监控俄大选期间网络活动 防其干涉内政
[军事] prediction:[军事(94.15%) 国内(3.87%) 文化(0.87%)] 空军首次颁发“空军飞行员光荣家庭”荣誉匾
[社会] prediction:[国内(55.30%) 社会(33.83%) 能源(7.74%)] 明起华北黄淮雾和霾来袭 南方降水增多
[国际] prediction:[国际(97.85%) 军事(1.77%) 财经(0.18%)] 美国扩大对朝鲜制裁 半岛局势持续呈紧张态势
[国内] prediction:[国内(56.21%) 能源(27.38%) 财经(14.75%)] 人民日报：去煤化”虽风起成